# Update routes & times
From time to time, CTtransit changes bus route ids. If the map stops working, download fresh GTFS feed and re-compile routes information.

https://www.cttransit.com/about/developers

In [2]:
import requests, zipfile, io
import pandas as pd
import json
import glob, os, os.path

### Download GTFS archive from CTtransit and unzip

In [3]:
r = requests.get("https://www.cttransit.com/sites/default/files/gtfs/googlect_transit.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="./")

### Filter only relevant routes

In [4]:
routes = pd.read_csv('./routes.txt')
routes_relevant = list(routes[ routes.route_short_name.isin(['37-39', '41', '61']) ].route_id)

trips = pd.read_csv('./trips.txt')
trips_relevant = trips[ trips.route_id.isin(routes_relevant) ]

### Save `trips.json` file

In [5]:
trips_relevant.filter([
    'route_id',
    'trip_id',
    'trip_headsign',
    'direction_id'
]).set_index('trip_id').to_json('./trips.json', orient='index')

### Remove all .txt files from `gtfs/`

In [6]:
for f in glob.glob("./*.txt"):
    os.remove(f)